In [1]:
import numpy as np
import pyvista as pv
import torch
import sys

In [ ]:
import torch
torch.cuda.is_available()

In [24]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int((size_bytes).bit_length() // 10)  # 根据大小选择合适的单位
    p = 1024 ** i
    size = round(size_bytes / p, 2)
    return f"{size} {size_name[i]}"

In [ ]:
size = int(1000000)  # 测试数据大小
parcel_numpy = np.random.rand(size, 9) * 1000

size_in_bytes = sys.getsizeof(parcel_numpy)  # 获取对象大小
print(f"Size of variable: {convert_size(size_in_bytes)}")

In [ ]:
import numpy as np
import torch
import timeit

# 初始化测试数据
def initialize_data(size):
    # 随机生成数据用于测试
    parcel_numpy = np.random.rand(size, 9) * 1000
    tStep = 0.01
    celllength = 0.5
    return parcel_numpy, tStep, celllength

# NumPy 实现的函数
def numpy_version(parcel, tStep, celllength):
    parcel[:, :3] += parcel[:, 3:6] * tStep
    i = np.floor((parcel[:, 0] / celllength) + 0.5).astype(int)
    j = np.floor((parcel[:, 1] / celllength) + 0.5).astype(int)
    k = np.floor((parcel[:, 2] / celllength) + 0.5).astype(int)
    parcel[:, 6] = i
    parcel[:, 7] = j
    parcel[:, 8] = k

# PyTorch 实现的函数
def pytorch_version(parcel, tStep, celllength, device='cpu'):
    parcel_tensor = torch.tensor(parcel, device=device)  # 将数据转换为PyTorch张量
    celllength_tensor = torch.tensor(celllength, device=device)

    parcel_tensor[:, :3] += parcel_tensor[:, 3:6] * tStep

    parcel_div_celllength = (parcel_tensor[:, :3] / celllength_tensor) + 0.5
    ijk = torch.floor(parcel_div_celllength).to(torch.int32)

    parcel_tensor[:, 6:9] = ijk

    return parcel_tensor.cpu().numpy()  # 返回到NumPy数组以便一致性

# 基准测试函数
def benchmark():
    size = int(1e8)  # 测试数据大小
    parcel_numpy, tStep, celllength = initialize_data(size)

    # 测试NumPy版本
    numpy_time = timeit.timeit(lambda: numpy_version(parcel_numpy.copy(), tStep, celllength), number=10)
    print(f'NumPy版本平均时间: {numpy_time / 10:.6f} 秒')

    # 测试PyTorch CPU版本
    pytorch_cpu_time = timeit.timeit(lambda: pytorch_version(parcel_numpy.copy(), tStep, celllength, device='cpu'), number=10)
    print(f'PyTorch CPU版本平均时间: {pytorch_cpu_time / 10:.6f} 秒')

    # 测试PyTorch GPU版本 (如果有GPU)
    if torch.cuda.is_available():
        pytorch_gpu_time = timeit.timeit(lambda: pytorch_version(parcel_numpy.copy(), tStep, celllength, device='cuda'), number=10)
        print(f'PyTorch GPU版本平均时间: {pytorch_gpu_time / 10:.6f} 秒')
    else:
        print('未检测到GPU，跳过GPU测试')

# 运行基准测试
if __name__ == "__main__":
    benchmark()


In [ ]:
import numpy as np
# import torch
import torch.utils.benchmark as benchmark

# 初始化测试数据
def initialize_data(size):
    # 随机生成数据用于测试
    parcel_numpy = np.random.rand(size, 9) * 1000
    tStep = 0.01
    celllength = 0.5
    return parcel_numpy, tStep, celllength

# NumPy 实现的函数
def numpy_version(parcel, tStep, celllength):
    parcel[:, :3] += parcel[:, 3:6] * tStep
    i = np.floor((parcel[:, 0] / celllength) + 0.5).astype(int)
    j = np.floor((parcel[:, 1] / celllength) + 0.5).astype(int)
    k = np.floor((parcel[:, 2] / celllength) + 0.5).astype(int)
    parcel[:, 6] = i
    parcel[:, 7] = j
    parcel[:, 8] = k

# PyTorch 实现的函数
def pytorch_version(parcel, tStep, celllength, device='cpu'):
    parcel_tensor = torch.tensor(parcel, device=device)  # 将数据转换为PyTorch张量
    celllength_tensor = torch.tensor(celllength, device=device)

    parcel_tensor[:, :3] += parcel_tensor[:, 3:6] * tStep

    parcel_div_celllength = (parcel_tensor[:, :3] / celllength_tensor) + 0.5
    ijk = torch.floor(parcel_div_celllength).to(torch.int32)

    parcel_tensor[:, 6:9] = ijk

    return parcel_tensor.cpu().numpy()  # 返回到NumPy数组以便一致性

# 基准测试函数
def benchmark_test():
    size = 1000000  # 测试数据大小
    parcel_numpy, tStep, celllength = initialize_data(size)

    # NumPy 基准测试
    t0 = benchmark.Timer(
        stmt="numpy_version(parcel_numpy.copy(), tStep, celllength)",
        globals={
            'numpy_version': numpy_version,
            'parcel_numpy': parcel_numpy,
            'tStep': tStep,
            'celllength': celllength
        },
        description="NumPy version"
    )

    # PyTorch CPU 基准测试
    t1 = benchmark.Timer(
        stmt="pytorch_version(parcel_numpy.copy(), tStep, celllength, device='cpu')",
        globals={
            'pytorch_version': pytorch_version,
            'parcel_numpy': parcel_numpy,
            'tStep': tStep,
            'celllength': celllength
        },
        description="PyTorch version (CPU)"
    )

    # PyTorch GPU 基准测试 (如果有GPU)
    if torch.cuda.is_available():
        t2 = benchmark.Timer(
            stmt="pytorch_version(parcel_numpy.copy(), tStep, celllength, device='cuda')",
            globals={
                'pytorch_version': pytorch_version,
                'parcel_numpy': parcel_numpy,
                'tStep': tStep,
                'celllength': celllength
            },
            description="PyTorch version (GPU)"
        )
    
    # 运行并打印基准结果
    print(t0.blocked_autorange(min_run_time=1))
    print(t1.blocked_autorange(min_run_time=1))

    if torch.cuda.is_available():
        print(t2.blocked_autorange(min_run_time=1))
    else:
        print("未检测到GPU，跳过GPU测试")

# 运行基准测试
if __name__ == "__main__":
    benchmark_test()


In [26]:

bottom = 900
height = 940

density = 10

sphere = np.ones((100, 100, 2000), dtype=bool)

radius = 30

center = 50


In [55]:
etching = np.load('./bosch_data_0924_ratio08/bosch_sf_step_5_sf.npy')

In [43]:
etching = np.load('./bosch_data_0924_ratio08/bosch_sf_step_1_c4f8.npy')

In [38]:
etching = np.load('./bosch_data_0924_ratio08/bosch_sf_step_0_Ar.npy')

In [22]:
etching = np.load('./bosch_sf_step10.npy')

In [ ]:
etching.shape

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-250:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-250:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-250:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-100:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-100:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-100:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()